In [1]:
from pathlib import Path # only supported in Python 3
import pandas as pd
import numpy as np
import pickle
import os

from tqdm import tqdm

In [2]:
task='psqm_newstest2020'

src='en'
trg='de'

lp_src = src + trg
lp = src + '-' + trg

In [ ]:
psqm_df = pd.read_csv("input\{}_{}.tsv".format(task, lp_src), sep='\t')
psqm_df.head()

In [ ]:
psqm_df.rename(columns = {'seg_id':'sid', 'source':'src', 'target':'mt'}, inplace = True)
psqm_df.head(1)

In [ ]:
print(psqm_df.dtypes)

#psqm_df['sid'] = psqm_df['sid'].astype(int) # no need
psqm_df['src'] = psqm_df['src'].astype('string')
psqm_df['mt'] = psqm_df['mt'].astype('string')

print(psqm_df.dtypes)

In [ ]:
all_model_names = list(psqm_df.system.unique())
all_model_names

In [ ]:
print(len(psqm_df.index))

psqm_df.dropna(subset=['src'], inplace=True)

print(len(psqm_df.index))

psqm_df.dropna(subset=['mt'], inplace=True)

print(len(psqm_df.index))

psqm_df.drop(psqm_df[psqm_df['src'].map(len) > 1000].index, inplace=True)

print(len(psqm_df.index))

psqm_df.drop(psqm_df[psqm_df['mt'].map(len) > 1000].index, inplace=True)

print(len(psqm_df.index))

In [ ]:
# Compute average scores

psqm_df['raw_score'] = 0.0

for model in all_model_names:
    
    print("------------------------ {} -----------------------".format(model))
    
    for idx in tqdm(list(all_ids)):
    
        rows = psqm_df.loc[(psqm_df['system'] == model) & (psqm_df['sid'] == idx)]
        print(rows['score'])

        n_rows = len(rows['score'])
        #print(n_rows)

        avg_score = np.mean(np.array(rows[['score']]), axis=0)[0]
        print("avg", avg_score)

        avg_score = avg_score * 100 / 7

        #print("avg norm", avg_score)

        psqm_df['raw_score'] = np.where((psqm_df['system'] == model) & (psqm_df['sid'] == idx), avg_score, psqm_df['raw_score'])
        
    
psqm_df.head(3)

In [ ]:
psqm_df.drop('rater', axis=1, inplace=True)
psqm_df.head(1)

In [ ]:
psqm_df.drop('score', axis=1, inplace=True)
psqm_df.head(1)

In [ ]:
print(len(psqm_df.index))

psqm_df.drop_duplicates(inplace=True)

print(len(psqm_df.index))

psqm_df.head(5)

In [ ]:
# placeholder values

psqm_df['bleu_score'] = 0
psqm_df['comet_score'] = 0

psqm_df.head(1)

## Print

In [ ]:
final_csv ="output\{}.csv".format(lp)
psqm_df.to_csv(final_csv, encoding="utf-8", index=False)

In [ ]:
new_psqm_df = pd.read_csv(final_csv)
new_psqm_df.head()

In [5]:
final_pickle = "output\{}\{}.pickle".format(task, lp)

In [ ]:
psqm_df.to_pickle(final_pickle)

In [6]:
filepath=Path(final_pickle)

data = []
with filepath.open('rb') as f:
    data = pickle.load(f)

new_psqm_df = pd.DataFrame(data)
new_psqm_df.head()

,system,doc,doc_id,sid,src,mt,raw_score,bleu_score,comet_score
0,eTranslation.737,independent.281139,1,1,Michael Jackson wore tape on his nose to get f...,"Michael Jackson trug Klebeband auf der Nase, u...",61.904762,0,0
3,Human-A.0,independent.281139,1,1,Michael Jackson wore tape on his nose to get f...,Ehemaliger Bodyguard berichtet: Michael Jackso...,80.952381,0,0
6,Human-B.0,independent.281139,1,1,Michael Jackson wore tape on his nose to get f...,"Ehemaliger Bodyguard behauptet, Michael Jackso...",71.428571,0,0
9,Human-P.0,independent.281139,1,1,Michael Jackson wore tape on his nose to get f...,"Um es aufs Cover zu schaffen, habe Michael Jac...",33.333333,0,0
12,Huoshan_Translate.832,independent.281139,1,1,Michael Jackson wore tape on his nose to get f...,"Michael Jackson trug Klebeband auf der Nase, u...",38.095238,0,0


In [8]:
all_ids = new_psqm_df.sid.unique()
print(all_ids)
print(len(all_ids))

[   1    2    3 ... 1416 1417 1418]
1417


In [9]:
with open("output\{}_ids.txt".format(lp), "w") as f:
    for el in all_ids:
        print(el, file=f)